# imports

In [2]:
import pandas as pd  
import numpy as np 
import os 

In [18]:
# webdriver: provides the API for accessing and controlling a web browser.
from selenium import webdriver

# Keys: provides keys that can be used for keyboard interactions.
from selenium.webdriver.common.keys import Keys

# By: provides mechanisms to locate elements on a web page.
from selenium.webdriver.common.by import By

# WebDriverWait: provides a way to wait for a specific condition to occur before proceeding with the code execution.
from selenium.webdriver.support.ui import WebDriverWait

# expected_conditions: provides a set of predefined conditions that can be used with WebDriverWait to wait for an element to become available, clickable, or visible.
from selenium.webdriver.support import expected_conditions as EC

In [24]:
from bs4 import BeautifulSoup

# Acquire Selenium

In [13]:
season_list = [1,2,2018,2019,2020,2021]

# Season 1 / 2015

In [56]:
# Create an instance of the Chrome browser
driver = webdriver.Chrome()

# Navigate to the BattleBots website
driver.get("https://battlebots.com/season-1-robots/")

# Get the page source
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


In [31]:
# Find all the <div> elements with class="cbp-wrapper"
div_elements = soup.find_all("div", {"class": "cbp-wrapper"})



In [45]:
links = []
# Find all the <div> elements with class="cbp-wrapper"
div_elements = soup.find_all("div", {"class": "cbp-wrapper"})

# Iterate over the list of matching <div> elements
for div in div_elements:
    # Find all the <div> elements with class="more-button has-love" within the current <div> element
    more_buttons = div.find_all("div", {"class": "more-button has-love"})
    
    # For example, you could extract the href attribute of the <a> element within each <div>
    for button in more_buttons:
        link = button.find("a")["href"]
        links.append(link)


# Test One Link
    

In [51]:
test = links[0]

In [62]:
# Navigate to the BattleBots website
driver.get(f'{test}')

# Get the page source
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

MaxRetryError: HTTPConnectionPool(host='localhost', port=64618): Max retries exceeded with url: /session/7e48af9525e042124f51b6faf4ec4528/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc94c7c19a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [66]:
tab = pd.read_html('https://battlebots.com/robot/beta/')

In [74]:
tab[0]

,Stats,Career,Season 2,Season 1
0,Total matches,4,4,0
1,Win percentage,75%,75%,NaN
2,Total wins,3,3,0
3,Losses,1,1,0
4,Knockouts,1,1,0
5,KO percentage,25%,25%,0%
6,Average knockout time,180s,180s,0
7,Knockouts against,0,0,0
8,KO against percentage,0%,0%,0%
9,Judges decision wins,2,2,0


In [75]:
tab[1]

,Season,Round,Opponent,Results
0,Season 2,Qualifier,Lucky,Won by JD (3-0)
1,Season 2,Round of 32,Overhaul,Won by JD (3-0)
2,Season 2,Round of 16,Nightmare,Won by KO (3:00)
3,Season 2,Quaterfinals,Tombstone,Lost by JD (0-3)
